In [ ]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pathlib
import PIL
from PIL import Image
from PIL import ImageOps
from PIL import ImageFilter

In [ ]:
test = pd.read_json("./dataset/IM_Test.json")
test_dir = pathlib.Path("./dataset/IM_Test/")
images = list(test_dir.glob('*.png'))

In [ ]:
model = keras.models.load_model("./models/imageProcessing/")
className = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
df = pd.DataFrame(columns=["imageid_boxid","class"])
for i in range(7000):
    bboxs = test.iloc[i]["bboxes"]
    filename = test.iloc[i]["filename"]
    image = ImageOps.grayscale(Image.open("dataset/IM_Test/"+filename).filter(ImageFilter.SHARPEN))
    for j,b in enumerate(bboxs) : 
        crop = image.crop((b.get("x1"),b.get("y1"),b.get("x2"),b.get("y2"))).resize((30,30))
        img_array = keras.utils.img_to_array(crop)
        img_array = tf.expand_dims(img_array, 0)
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        cl = className[np.argmax(score)]
        print("File",filename,className[np.argmax(score)] ,j)
        da = pd.DataFrame([[str(i+1)+"_"+str(j),className[np.argmax(score)]]],columns=["imageid_boxid","class"])
        df = pd.concat([df,da])
        if(i % 200 == 0): df.to_csv("./submit/a.csv", index=False)
df.to_csv("./submit/a.csv", index=False)